In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

c:\Users\3dsma\miniconda3\envs\torch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [38]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 5) #input -? OUtput? RF
        self.conv1_bn = nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8, 16, 3, padding=1)
        self.conv2_bn = nn.BatchNorm2d(16)
        self.conv3 = nn.Conv2d(16, 32, 3, padding=1)
        self.conv3_bn = nn.BatchNorm2d(32)
        self.conv4 = nn.Conv2d(32, 32, 1)
        self.conv4_bn = nn.BatchNorm2d(32)
        self.fc1 = nn.Linear(in_features=288, out_features=40)
        self.fc1_bn = nn.BatchNorm1d(40)
        self.fc2 = nn.Linear(in_features=40, out_features=20)
        self.fc2_bn = nn.BatchNorm1d(20)
        self.out = nn.Linear(in_features=20, out_features=10)

        self.dropout = nn.Dropout(0.05)


    def forward(self, x):
        x = F.relu(self.conv1_bn(self.dropout(self.conv1(x)))) # 28 |24
        x = F.relu(self.conv2_bn(self.dropout(self.conv2(x))))# 24 | 24
        x = F.max_pool2d(x, kernel_size=2, stride=2) # 24 | 12
        x = F.relu(self.conv3_bn(self.dropout(self.conv3(x)))) # 12 | 12
        x = F.max_pool2d(x, kernel_size=2, stride=2) # 12 | 6
        x = F.relu(self.conv4_bn(self.dropout(self.conv4(x)))) # 6 | 6
        x = F.max_pool2d(x, kernel_size=2, stride=2) # 6 | 3
        x = x.flatten(1) # 32 * 3 * 3 = 288
        x = F.relu(self.fc1_bn(self.dropout(self.fc1(x)))) # 288 | 40
        x = F.relu(self.fc2_bn(self.dropout(self.fc2(x)))) # 40 | 20
        x = self.out(x) # 20 | 10
        x = x.view(-1, 10)
        return F.log_softmax(x, dim=1)

In [39]:
# !pip install torchinfo
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 24, 24]             208
           Dropout-2            [-1, 8, 24, 24]               0
       BatchNorm2d-3            [-1, 8, 24, 24]              16
            Conv2d-4           [-1, 16, 24, 24]           1,168
           Dropout-5           [-1, 16, 24, 24]               0
       BatchNorm2d-6           [-1, 16, 24, 24]              32
            Conv2d-7           [-1, 32, 12, 12]           4,640
           Dropout-8           [-1, 32, 12, 12]               0
       BatchNorm2d-9           [-1, 32, 12, 12]              64
           Conv2d-10             [-1, 32, 6, 6]           1,056
          Dropout-11             [-1, 32, 6, 6]               0
      BatchNorm2d-12             [-1, 32, 6, 6]              64
           Linear-13                   [-1, 40]          11,560
          Dropout-14                   

In [40]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [41]:
# !pip install tqdm

In [42]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [43]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=0.08818554133176804 batch_id=468: 100%|██████████| 469/469 [00:11<00:00, 41.86it/s] 



Test set: Average loss: 0.0586, Accuracy: 9845/10000 (98%)



loss=0.023718656972050667 batch_id=468: 100%|██████████| 469/469 [00:10<00:00, 43.04it/s]



Test set: Average loss: 0.0398, Accuracy: 9881/10000 (99%)



loss=0.030311210080981255 batch_id=468: 100%|██████████| 469/469 [00:11<00:00, 42.61it/s] 


...


Test set: Average loss: 0.0223, Accuracy: 9932/10000 (99%)



loss=0.08907725661993027 batch_id=468: 100%|██████████| 469/469 [00:11<00:00, 42.48it/s]  



Test set: Average loss: 0.0234, Accuracy: 9935/10000 (99%)



loss=0.06590617448091507 batch_id=468: 100%|██████████| 469/469 [00:11<00:00, 42.38it/s]   



Test set: Average loss: 0.0226, Accuracy: 9925/10000 (99%)

